In [ ]:
"""
Compute classical population genetic neutrality statistics (Tajima's D, Fu & Li's D, 
Fay & Wu's H, Zeng's E) in sliding windows from SLiM simulation outputs.
"""

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nrand
import scipy.stats as stats
import pickle as pkl
import seaborn as sns
import itertools

In [ ]:
"""
Define helper functions for harmonic-number–based constants and each neutrality statistic.
"""

def scaling_factor(n):
    """
    Compute a1 = sum_{i=1}^{n-1} 1/i.
    """
    a = 0
    if n == 20000: #  return precomputed value for large n to save time.
        return 10.48067821722932 
    for i in range(1, n):
        a += 1/i
    return a

def get_a2(n):
    """
    Compute a2 = sum_{i=1}^{n-1} 1/i^2.
    """
    a = 0
    if n == 20000: #  return precomputed value for large n to save time.
        return 1.6448840655982087
    elif n == 20001:
        return 1.6448840680982086
    for i in range(1, n):
        a += 1/(i**2)
    return a

def get_TajimasD(mut_df, Ne):
    n = 2*Ne
    a1 = scaling_factor(n)
    a2 = get_a2(n)
    b1 = (n+1)/(3*(n-1))
    b2 = 2*(n**2+n+3)/(9*n*(n-1))
    c1 = b1-1/a1
    c2 = b2-(n+2)/(a1*n)+a2/(a1**2)
    e1 = c1/a1
    e2 = c2/(a1**2+a2)
    AF = mut_df['AF']/(2*Ne)
    Tpi = 2*AF*(1-AF)
    Tpi_sum = np.sum(Tpi)
    S = len(mut_df)
    TW = S/scaling_factor(2*Ne)
    D = (Tpi_sum - TW)/np.sqrt(e1*S+e2*S*(S-1))
    return D

def get_FuLisD(mut_df, Ne):
    n = 2*Ne
    S = len(mut_df)
    Se = (mut_df['AF'] == 1).sum()
    an = scaling_factor(n)
    bn = get_a2(n)
    cn = 2*(n*an-2*(n-1))/(n-1)/(n-2)
    vd = 1+an**2/(bn+an**2)*(cn-(n+1)/(n-1))
    ud = an - 1 - vd
    D = (S - an*Se)/np.sqrt(ud*S+vd*S**2)
    return D

def get_FayWusH(mut_df, Ne):
    n = 2*Ne
    TL = mut_df['AF'].sum()/(n-1)
    AF = mut_df['AF']/n
    Tpi = 2*AF*(1-AF)
    Tpi_sum = np.sum(Tpi)
    S = len(mut_df)
    an = scaling_factor(n)
    bn = get_a2(n)
    bn_1 = get_a2(n+1)
    T = S/an
    T2 = S*(S-1)/(an**2+bn)
    var = T*(n-2)/(6*(n-1)) + T2*(18*n**2*(3*n+2)*bn_1-(88*n**3+9*n**2-13*n+6))/(9*n*(n-1)**2)
    H = (Tpi_sum - TL)/np.sqrt(var)
    return H

def get_ZengsE(mut_df, Ne):
    n = 2*Ne
    an = scaling_factor(n)
    bn = get_a2(n)
    S = len(mut_df)
    TL = mut_df['AF'].sum()/(2*Ne-1)
    TW = S/an
    T2 = S*(S-1)/(an**2+bn)
    var = TW*(n/(2*(n-1))-1/an) + T2*(bn/an**2+2*bn*(n/(n-1))**2-2*(n*bn-n+1)/(an*(n-1))-(3*n+1)/(n-1))
    E = (TL - TW)/np.sqrt(var)
    return E

In [ ]:
"""
For each simulation model ("version") and replicate:
   - Load mutation data from text files (allele frequencies, positions, etc.).
   - Slide a window (10kb wide, shifted 1kb each step) across the genome.
   - Compute statistics for each window and collect them.

Inputs
------
./data/Simulation_selsweep/rep<rep>/<version>_20samples/<version>_<N_gen>.txt
    Text tables from simulations. Must include:
    - ID  (mutation identifier)
    - pos (genomic position)
    - s   (selection coefficient)
    - SG  (generation when mutation arose)
    - AF  (allele frequency in counts, not normalized)
"""

# Parameters for windowed analysis
sliding_size = 1000
window_size = 10000
Ne = 10000

# Containers for results
TajimasD_dict = {}
FuLisD_dict = {}
FayWusH_dict = {}
ZengsE_dict = {}

# Iterate over models and replicates
for version in ['Pseudo', 'Neutral', 'AdapTrack', 'AdapTrack_env20', 'Adaptive']:
    print(version)
    TajimasD_list = []
    FuLisD_list = []
    FayWusH_list = []
    ZengsE_list = []
    for rep in range(1, 31):
        print(rep, end='\r', flush=True)
        target_dir = \
            f'./data/Simulation_selsweep/rep{rep}/{version}_20samples/'   
        for N_gen in range(80000, 100001, 1000): # generations 80k–100k, step 1k
            print(N_gen, end='\r', flush=True)
            mut_df = pd.read_table(target_dir+f'{version}_{N_gen}.txt', header=None, sep='\s+')
            
            # keep only relevant columns
            mut_df = mut_df.rename(columns={4:'ID', 6:'pos', 7:'s', 10:'SG', 11:'AF'})

            # Slide windows across genome (100 kb total length assumed from code)
            for i in range(int((100000-window_size)/sliding_size)+1):
                mut_df_sub = mut_df[
                    (i*sliding_size<mut_df['pos']) & (mut_df['pos'] < i*sliding_size+window_size)
                ]

                # Compute statistics on this window
                TajimasD = get_TajimasD(mut_df_sub, Ne)
                FuLisD = get_FuLisD(mut_df_sub, Ne)
                FayWusH = get_FayWusH(mut_df_sub, Ne)
                ZengsE = get_ZengsE(mut_df_sub, Ne)
                TajimasD_list.append(TajimasD)
                FuLisD_list.append(FuLisD)
                FayWusH_list.append(FayWusH)
                ZengsE_list.append(ZengsE)
    
    # Store results
    TajimasD_dict[version] = TajimasD_list
    FuLisD_dict[version] = FuLisD_list
    FayWusH_dict[version] = FayWusH_list
    ZengsE_dict[version] = ZengsE_list


In [ ]:
# Uncomment and run to overwrite result files

# with open('./data/TajimasD_dict.pkl', 'wb') as f:
#     pkl.dump(TajimasD_dict, f)
# with open('./data/FuLisD_dict.pkl', 'wb') as f:
#     pkl.dump(FuLisD_dict, f)
# with open('./data/FayWusH_dict.pkl', 'wb') as f:
#     pkl.dump(FayWusH_dict, f)
# with open('./data/ZengsE_dict.pkl', 'wb') as f:
#     pkl.dump(ZengsE_dict, f)
